## 🎬 영화 리뷰 데이터로 BERT 모델 감성 분석하도록 훈련 시키기

### 📂 Load dataset

- huggingface `datasets` package 
  ```
    pip install datasets
  ```

- nsmc dataset
  - https://huggingface.co/datasets/nsmc

In [4]:
from datasets import load_dataset

nsmc_dataset = load_dataset("nsmc")

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
nsmc_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [23]:
#####################################################################################
# DataSetDict 형태로 데이터 구조 , 내용 살펴보기
#####################################################################################

# document 는 리뷰 , label 은 리뷰의 긍정/부정 (positive / negative)
display(nsmc_dataset['train'].features)

# label 의 명칭은 'negative' , 'positive' 인데 여기서는 0, 1 로 표현되어 있음
display(nsmc_dataset['train'][0])

# label 명에 따른 label id 는 `label` feature 를 str2int 로 확인 
display(nsmc_dataset['train'].features['label'].str2int('positive'), 
        nsmc_dataset['train'].features['label'].str2int('negative'))

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

1

0

In [24]:
#####################################################################################
# 판다스 데이터 프레임으로 변환해서 좀더 세밀히 살펴보기
#####################################################################################

# pandas 데이터 프레임으로 변환해서 자세히 살펴보기
nsmc_df = nsmc_dataset['train'].to_pandas()
display(nsmc_df.head())

# (⚠️ 주의)분류 문제인 경우 label 불균등을 확인해보는 것이 중요
# - 만일 label 이 매우 불균등한 상태라면 over sampling, under sampling 등 적용 필요
# - negative 50.1% , positive 49.9%
display(nsmc_df.groupby('label').apply(lambda x:len(x)/len(nsmc_df)))

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


/tmp/ipykernel_1702/2225452072.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  display(nsmc_df.groupby('label').apply(lambda x:len(x)/len(nsmc_df)))


label
0    0.501153
1    0.498847
dtype: float64

In [25]:
# 리뷰 문자열 길이 알아보기
nsmc_df['review_length'] = nsmc_df['document'].str.len()
nsmc_df['review_length'].describe()

count    150000.000000
mean         35.203353
std          29.532097
min           0.000000
25%          16.000000
50%          27.000000
75%          42.000000
max         146.000000
Name: review_length, dtype: float64

#### 📂 preprocess

자연어 문제의 기본은 `Tokenization` 으로 매우 중요. 예를 들어, `Tokenization` 의 방식에 따라 의미가 달라짐

여기서는 BERT Multilangual Tokenizer 를 그대로 사용할 것 (BERT 모델에 들어 있는 것) 

`Auto` 붙은 것은 pretrained 모델을 갖고 올 때 그에 맞는 class 를 retrieval 해주는 것일 뿐

모델은 https://huggingface.co/google-bert/bert-base-multilingual-cased  이 것 가지고 올 것임

※ 한국어 Tokenizer , KoBERT 등 사용하는 것도 괜찮음


In [28]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tok


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [35]:
#####################################################################################
# 문장 tokenize 해보기
# - 한국어만으로 학습된 tokenizer 가 아닌 다국어로 학습된 것임에도 어느정도는 잘 함
# - `##` 이 들어가지 않은 것은 단어가 시작되는 부분, 들어간 것은 이전 단어와 이어지는 것
#####################################################################################
seq = "청춘 영화의 최고봉."

tok.tokenize(seq)

['청', '##춘', '영화', '##의', '최고', '##봉', '.']

In [41]:
# input_ids 는 토큰을 vocabulary 에 포함된 토큰이 id 로 매핑한 결과
# attention_mask 는 해당 토큰이 실제 데이터인지 여부를 마킹 (패팅 토큰 등은 0)
# token_type_ids 는 문장을 구분하는데 사용됨. 
# - 예를 들어 premise, hypothesis 의 NLI (함의/모순/중립 판단) 하는 경우 두 개의 문장을 서로 구분하는 역할
from pprint import pprint
pprint(tok(seq))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 9751, 97707, 42428, 10459, 83491, 118989, 119, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [44]:
# 패딩으로 채워진 두 번째 짧은 문장에서 패팅에 해당 하는 위치는 attention mask 가 0 임
seq2 = '청춘'
pprint(tok([seq, seq2], padding=True))

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0]],
 'input_ids': [[101, 9751, 97707, 42428, 10459, 83491, 118989, 119, 102],
               [101, 9751, 97707, 102, 0, 0, 0, 0, 0]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]}


### 참고

NLI Task 의 경우 아래 같은 코드를 통해 입력을 모델에게 줌 (예시임. GPT)

In [42]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

premise = "The quick brown fox jumps over the lazy dog. It did not stop running."
hypothesis = "A fast animal is moving."

input_text = "[CLS] " + premise + " [SEP] " + hypothesis + " [SEP]"
tokenized_output = tokenizer.tokenize(input_text)

input_ids = tokenizer.convert_tokens_to_ids(tokenized_output)

# 🔑 [CLS] , [SEP] 이 포함된 모든 토큰에 대해 `1` 로 설정 
attention_mask = [1] * len(input_ids) 

# 🔑 premise 는 0 , hypothesis 는 1 로 설정 
sep_indices = [i for i, token in enumerate(tokenized_output) if token == "[SEP]"]
token_type_ids = [0 if i <= sep_indices[0] else 1 for i in range(len(tokenized_output))]


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]